In [37]:
import pandas as pd 
import numpy as np
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

names = ['parents', 'h_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'target']
df = pd.read_csv('C:/Users/LAPPY/Desktop/DDEEPP/Dm_ass4/nursery.data', sep=',', header=None)
for i in range(len(names)):
    df[i] = df[i].astype('category')
df.columns = names

In [38]:
labelencoder = LabelEncoder()

for i in range(len(df.columns)):
    df.iloc[:,i] = labelencoder.fit_transform(df.iloc[:,i])


In [39]:
train, test = train_test_split(df, test_size=0.2)
# print(len(train))
# print(len(test))

y_test = list(test.iloc[:,-1])
test = test.iloc[:,:-1]

df = train

In [40]:
y = list(df.iloc[:,-1])

def calculate_y_prob():
    global y
    class_dict=dict()
    
    for i in y:                 #finding count of every class in target column
        if i in class_dict:
            class_dict[i]+=1
        else:
            class_dict[i]=1
            
    y_count = class_dict.copy()
    
    n = len(y)
    for k in class_dict:
        class_dict[k]/=n                            #calculating prior prob p(y)  stored in dictionary  classwise

    return y_count,class_dict

y_count,y_prob = calculate_y_prob()
y_classes = sorted(list(y_prob.keys()))


In [41]:

def feature_prob():
    global y
    global df
    global y_classes
    global y_count
    
    num_classes = len(y_classes)
    features_tables_list = []
    n = len(df)
    
    n_cols = len(df.columns) - 1
#     print(n_cols)
    for i in range(n_cols):
#         print(i)
        col = list(df.iloc[:,i])
        num_values = len(set(col))
        df_feat = pd.DataFrame(np.zeros(num_classes*num_values).reshape(num_values,num_classes))
#         print(df_feat)
#calculating count of feature value per class         
        for j in range(n):
#             print('col',col[j])
#             print('y',y[j])
            df_feat.iloc[col[j]][y[j]]+=1
#calculating probabilty by dividing with count         
        for j in range(num_classes):
            df_feat.iloc[:,j] /= y_count[j]
        
        print(df_feat)
        features_tables_list.append(df_feat)
        
    return features_tables_list
    
features_prob_list = feature_prob()

          0         1    2         3         4
0  0.334390  0.194923  0.0  0.503566  0.000000
1  0.330931  0.346659  0.0  0.311008  0.391837
2  0.334679  0.458418  1.0  0.185426  0.608163
          0         1    2         3         4
0  0.197175  0.111176  0.0  0.312558  0.000000
1  0.202940  0.211555  0.0  0.186667  0.208163
2  0.204958  0.312810  0.0  0.065426  0.395918
3  0.196022  0.316020  1.0  0.062326  0.395918
4  0.198905  0.048439  0.0  0.373023  0.000000
          0         1    2         3         4
0  0.243010  0.275751  1.0  0.209612  0.371429
1  0.249640  0.259119  0.0  0.245891  0.302041
2  0.256270  0.227896  0.0  0.284031  0.126531
3  0.251081  0.237234  0.0  0.260465  0.200000
          0         1    2         3         4
0  0.255982  0.289466  1.0  0.197519  0.440816
1  0.253675  0.253283  0.0  0.237209  0.302041
2  0.244163  0.230231  0.0  0.280620  0.126531
3  0.246180  0.227021  0.0  0.284651  0.130612
          0         1    2         3         4
0  0.334102  

In [42]:
def classify(test_point):
    global y_classes
    global y_prob
    global features_prob_list
    
    _prob = [ ]
    
    for y in y_classes:
        p = y_prob[y]
        for i in range(len(test_point)):
            #print(features_prob_list[i])
            p*=features_prob_list[i].iloc[test_point[i]][y]
        _prob.append(p)
    
    
    maxIn = -1
    maxVal = -1
    for i in range(len(_prob)):
        if _prob[i]>maxVal:
            maxVal=_prob[i]
            maxIn=i
            
    return y_classes[maxIn]

In [43]:
pred = []
for i in range(len(test)):
    pred.append(classify(list(test.iloc[i])))
    

In [44]:
acc = 0
for i in range(len(test)):
    if pred[i] == y_test[i]:
        acc+=1
print(acc/len(test)*100)
# print(len(pred))
# print(len(y_test))

90.2006172839506


In [45]:

conf_mat = pd.DataFrame(np.zeros(len(y_classes)**2).reshape(len(y_classes),len(y_classes)))
# print(conf_mat)
#y_test

for i in range(len(y_test)):
    conf_mat.iloc[pred[i],y_test[i]]+=1
#         print(conf_mat)

print('rows = predicted values\ncolumns = actual values\n')
print(conf_mat)

rows = predicted values
columns = actual values

       0      1    2      3     4
0  851.0    0.0  0.0    0.0   0.0
1    0.0  760.0  0.0   97.0  78.0
2    0.0    0.0  0.0    0.0   0.0
3    0.0   79.0  0.0  722.0   0.0
4    0.0    0.0  0.0    0.0   5.0


In [46]:
precision = np.zeros(len(y_classes))
for i in range(len(y_classes)):
    t = list(conf_mat[i])
    num = t[i]
    den = sum(t)
    try:
        precision[i] = num/den
    except:
        pass
print(precision)

[1.         0.90584029 0.         0.88156288 0.06024096]


In [47]:
recall = np.zeros(len(y_classes))
for i in range(len(y_classes)):
    t = list(conf_mat.iloc[i])
    num = t[i]
    den = sum(t)
    try:
        recall[i] = num/den
    except:
        pass
print(recall)

[1.         0.81283422 0.         0.90137328 1.        ]
